In [1]:
from config import set_path

set_path()

import sys
print(sys.version)


3.12.3 (main, Feb  4 2025, 14:48:35) [GCC 13.3.0]


In [2]:
from core import load_data

In [3]:
data = load_data('parquet')

In [4]:
data.sample(5)

,excerto,categoria
2977,Programa Contrata SP - Pessoa 60+. Criar e rea...,Desenvolvimento Econômico e Trabalho
889,E com relação principalmente a esse Centro de ...,Pessoa com Deficiência
3418,Viabilização Parque Linear do Bananal - subpre...,Meio Ambiente
780,"CAPS, tem aqui a proposta de 6 CAPS. Eu acho t...",Saúde
1010,A gente também pensa em criar programa de capa...,Saúde


In [5]:
data['categoria'].value_counts()

categoria
Saúde                                     546
Transportes e Mobilidade                  472
Zeladoria Urbana e melhorias de bairro    462
Desenvolvimento Econômico e Trabalho      385
Outros temas                              358
Assistência Social                        289
Meio Ambiente                             278
Habitação                                 239
Educação                                  211
Cultura                                   157
Segurança Urbana                          141
Direitos Humanos e Cidadania              129
Infraestrutura Urbana e Obras             124
Urbanismo e Licenciamento                 121
Esporte e Lazer                            63
Inovação e Tecnologia                      55
Pessoa com Deficiência                     32
Name: count, dtype: int64

In [6]:
menos_que_100 = data['categoria'].value_counts().index[data['categoria'].value_counts()<100]

In [7]:
menos_que_100

Index(['Esporte e Lazer', 'Inovação e Tecnologia', 'Pessoa com Deficiência'], dtype='object', name='categoria')

In [8]:
data['categoria'] = data['categoria'].apply(lambda x: 'Outros temas' if x in menos_que_100 else x)

In [9]:
data['categoria'].value_counts()<100

categoria
Saúde                                     False
Outros temas                              False
Transportes e Mobilidade                  False
Zeladoria Urbana e melhorias de bairro    False
Desenvolvimento Econômico e Trabalho      False
Assistência Social                        False
Meio Ambiente                             False
Habitação                                 False
Educação                                  False
Cultura                                   False
Segurança Urbana                          False
Direitos Humanos e Cidadania              False
Infraestrutura Urbana e Obras             False
Urbanismo e Licenciamento                 False
Name: count, dtype: bool

In [10]:
import nltk

In [11]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/h-pgy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/h-pgy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/h-pgy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
!python -m spacy download pt_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.2/568.2 MB 11.4 MB/s eta 0:00:00m eta 0:00:010:00:01
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')


In [13]:

from nltk.corpus import stopwords
import re
import spacy

class Preprocesser:

    my_stopwords = {
        'prefeito',
        'prefeitura',
        'subprefeitura',
        'bruno covas',
        'programa de metas',
        'iniciativa',
        'meta',
        'bom dia',
        'boa tarde',
        'boa noite',
        'olá',
        'oi',
        'revisar',
        'ampliar'
        }

    def __init__(self):

        self.custom_stopwords = set(stopwords.words('portuguese')).union(
                            self.my_stopwords)
        
        # Carregar modelo do spaCy para português
        self.spacy_nlp = spacy.load("pt_core_news_lg")

        
    
    # Função de pré-processamento
    def preprocess_text(self, text:str):
        text = text.lower()  # Converter para minúsculas
        text = re.sub(r'\d+', '', text)  # Remover números
        text = re.sub(r'\W+', ' ', text)  # Remover caracteres especiais
        doc = self.spacy_nlp(text)  # Processar texto com spaCy
        tokens = [token.lemma_ for token in doc if token.text not in self.custom_stopwords and not token.is_punct]
        return ' '.join(tokens)
    
    def __call__(self, text:str)->str:

        return self.preprocess_text(text)

In [14]:
preprocess_text = Preprocesser()

In [15]:
data['processed_text'] = data['excerto'].apply(lambda x: preprocess_text(x))

In [16]:
data.sample(5)

,excerto,categoria,processed_text
3674,A gente até sabe que há uma vontade do Prefei...,Zeladoria Urbana e melhorias de bairro,gente saber vontade fazer subprefeito novo e...
22,"220 mil Regularização fundiária, meta de númer...",Habitação,mil regularização fundiário número último pl...
1155,"Outra questão importante e fundamental, que é ...",Direitos Humanos e Cidadania,outro questão importante fundamental vacinação...
2992,Implantar parques lineares nos córregos com fi...,Meio Ambiente,implantar parque linear córrego financiamento ...
2994,Inclusão digital a pessoas 60+. Promover capac...,Outros temas,inclusão digital pessoa promover capacitação a...


In [17]:
data['excerto'].duplicated().any()

np.False_

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.pipeline import Pipeline

In [19]:
# Separar features e labels
X_train, X_test, y_train, y_test = train_test_split(data['processed_text'], data['categoria'], test_size=0.2, random_state=42)

# Criar pipeline com TF-IDF e SVC
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(min_df=3, max_df=0.85, ngram_range=(1,2))),
    ('svc', SVC(kernel='linear', random_state=42))
])

# Treinar o modelo
pipeline.fit(X_train, y_train)

# Avaliação
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")


                                        precision    recall  f1-score   support

                    Assistência Social       0.77      0.78      0.78        60
                               Cultura       0.80      0.71      0.75        34
  Desenvolvimento Econômico e Trabalho       0.75      0.72      0.74        89
          Direitos Humanos e Cidadania       0.64      0.47      0.55        19
                              Educação       0.82      0.73      0.77        44
                             Habitação       0.76      0.74      0.75        34
         Infraestrutura Urbana e Obras       0.67      0.38      0.49        26
                         Meio Ambiente       0.78      0.75      0.76        51
                          Outros temas       0.54      0.75      0.63        99
                                 Saúde       0.80      0.81      0.80       103
                      Segurança Urbana       0.83      0.53      0.65        19
              Transportes e Mobilidade 

In [20]:
from sklearn.model_selection import GridSearchCV

# Definir os hiperparâmetros a serem testados
param_grid = {
    'svc__C': [0.01, 0.1, 1, 10, 100],  # Regularização
    'svc__kernel': ['linear', 'rbf', 'poly'],  # Tipos de kernel
    'svc__gamma': [1e-3, 1e-2, 0.1, 1],  # Parâmetro usado para kernels não lineares
    'svc__class_weight': ['balanced', None]
}

# Criar GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', verbose=2, n_jobs=-1)

# Executar a busca
grid_search.fit(X_train, y_train)





Fitting 5 folds for each of 120 candidates, totalling 600 fits
[CV] END svc__C=0.01, svc__class_weight=balanced, svc__gamma=0.001, svc__kernel=linear; total time=   3.8s
[CV] END svc__C=0.01, svc__class_weight=balanced, svc__gamma=0.001, svc__kernel=linear; total time=   3.8s
[CV] END svc__C=0.01, svc__class_weight=balanced, svc__gamma=0.001, svc__kernel=poly; total time=   3.9s
[CV] END svc__C=0.01, svc__class_weight=balanced, svc__gamma=0.001, svc__kernel=linear; total time=   3.9s
[CV] END svc__C=0.01, svc__class_weight=balanced, svc__gamma=0.001, svc__kernel=poly; total time=   3.8s
[CV] END svc__C=0.01, svc__class_weight=balanced, svc__gamma=0.001, svc__kernel=linear; total time=   4.0s
[CV] END svc__C=0.01, svc__class_weight=balanced, svc__gamma=0.001, svc__kernel=linear; total time=   4.1s
[CV] END svc__C=0.01, svc__class_weight=balanced, svc__gamma=0.001, svc__kernel=rbf; total time=   4.0s
[CV] END svc__C=0.01, svc__class_weight=balanced, svc__gamma=0.001, svc__kernel=rbf; tot

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(max_df=0.85, min_df=3,
                                                        ngram_range=(1, 2))),
                                       ('svc',
                                        SVC(kernel='linear',
                                            random_state=42))]),
             n_jobs=-1,
             param_grid={'svc__C': [0.01, 0.1, 1, 10, 100],
                         'svc__class_weight': ['balanced', None],
                         'svc__gamma': [0.001, 0.01, 0.1, 1],
                         'svc__kernel': ['linear', 'rbf', 'poly']},
             scoring='accuracy', verbose=2)

In [21]:
# Melhor combinação de hiperparâmetros
print("Melhores parâmetros:", grid_search.best_params_)
print(f"Melhor acurácia: {grid_search.best_score_:.4f}")

Melhores parâmetros: {'svc__C': 10, 'svc__class_weight': 'balanced', 'svc__gamma': 0.1, 'svc__kernel': 'rbf'}
Melhor acurácia: 0.7239


In [27]:

# Avaliação no conjunto de teste
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print(classification_report(y_test, y_pred))
print(f"Accuracy no teste: {accuracy_score(y_test, y_pred):.4f}")
print(f"F1 no teste: {f1_score(y_test, y_pred, average='micro'):.4f}")

                                        precision    recall  f1-score   support

                    Assistência Social       0.73      0.78      0.76        60
                               Cultura       0.81      0.76      0.79        34
  Desenvolvimento Econômico e Trabalho       0.77      0.73      0.75        89
          Direitos Humanos e Cidadania       0.50      0.58      0.54        19
                              Educação       0.74      0.73      0.74        44
                             Habitação       0.79      0.76      0.78        34
         Infraestrutura Urbana e Obras       0.45      0.38      0.42        26
                         Meio Ambiente       0.69      0.73      0.70        51
                          Outros temas       0.57      0.71      0.63        99
                                 Saúde       0.86      0.76      0.80       103
                      Segurança Urbana       0.83      0.53      0.65        19
              Transportes e Mobilidade 

In [29]:
print(f"F1 no teste com peso por classe: {f1_score(y_test, y_pred, average='weighted'):.4f}")

F1 no teste com peso por classe: 0.7201
